1,2.Import pyspark

In [51]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySparkass").getOrCreate()

24/09/26 17:19:02 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


3. Load the CSV file as a Spark RDD


In [42]:
df = spark.read.csv("/home/ai/Documents/Task_hotel/hotels.csv",header=True)
rdd1=df.rdd
print(type(rdd1))
df=rdd1.toDF()
df.show()

<class 'pyspark.rdd.RDD'>
+---------------+--------------+---------+----------------+----------+----------+--------+--------------------+--------------------+
|Holding company|  Headquarters|Locations|Year established|Properties|     Rooms|Category|      Managed brands|   Franchised brands|
+---------------+--------------+---------+----------------+----------+----------+--------+--------------------+--------------------+
|          Accor|Paris (France)|Worldwide|            1967|  5,300[4]|778,000[4]|  Luxury|             Raffles|Banyan Tree Hótel...|
|           NULL|          NULL|     NULL|            NULL|      NULL|      NULL|    NULL|      Orient Express|                NULL|
|           NULL|          NULL|     NULL|            NULL|      NULL|      NULL|    NULL|              Delano|                NULL|
|           NULL|          NULL|     NULL|            NULL|      NULL|      NULL|    NULL|                 SO/|                NULL|
|           NULL|          NULL|     NULL| 

4.1.1 creating first dataframe

In [43]:
df1 = df[["Holding company", "Headquarters", "Locations", "Year established", "Properties", "Rooms"]]
df1.show()

+---------------+--------------+---------+----------------+----------+----------+
|Holding company|  Headquarters|Locations|Year established|Properties|     Rooms|
+---------------+--------------+---------+----------------+----------+----------+
|          Accor|Paris (France)|Worldwide|            1967|  5,300[4]|778,000[4]|
|           NULL|          NULL|     NULL|            NULL|      NULL|      NULL|
|           NULL|          NULL|     NULL|            NULL|      NULL|      NULL|
|           NULL|          NULL|     NULL|            NULL|      NULL|      NULL|
|           NULL|          NULL|     NULL|            NULL|      NULL|      NULL|
|           NULL|          NULL|     NULL|            NULL|      NULL|      NULL|
|           NULL|          NULL|     NULL|            NULL|      NULL|      NULL|
|           NULL|          NULL|     NULL|            NULL|      NULL|      NULL|
|           NULL|          NULL|     NULL|            NULL|      NULL|      NULL|
|           NULL

4.1.2 dealing null values with forward fill

In [44]:
import pyspark.sql.functions as F
from pyspark.sql import Window
w_forward = Window.partitionBy().rowsBetween(Window.unboundedPreceding,Window.currentRow)
df1 = df1.select([F.last(df1[column], ignorenulls=True).over(w_forward).alias(column) for column in df1.columns])
df1.show()

24/09/26 17:18:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------------+--------------+---------+----------------+----------+----------+
|Holding company|  Headquarters|Locations|Year established|Properties|     Rooms|
+---------------+--------------+---------+----------------+----------+----------+
|          Accor|Paris (France)|Worldwide|            1967|  5,300[4]|778,000[4]|
|          Accor|Paris (France)|Worldwide|            1967|  5,300[4]|778,000[4]|
|          Accor|Paris (France)|Worldwide|            1967|  5,300[4]|778,000[4]|
|          Accor|Paris (France)|Worldwide|            1967|  5,300[4]|778,000[4]|
|          Accor|Paris (France)|Worldwide|            1967|  5,300[4]|778,000[4]|
|          Accor|Paris (France)|Worldwide|            1967|  5,300[4]|778,000[4]|
|          Accor|Paris (France)|Worldwide|            1967|  5,300[4]|778,000[4]|
|          Accor|Paris (France)|Worldwide|            1967|  5,300[4]|778,000[4]|
|          Accor|Paris (France)|Worldwide|            1967|  5,300[4]|778,000[4]|
|          Accor

4.1.3 only 1 row per "Holding company"

In [45]:
from pyspark.sql import functions as F
df_first = df1.groupBy("Holding company").agg(
     F.first("Headquarters").alias("Headquarters"),
     F.concat_ws(", ", F.collect_list("Locations")).alias("Locations"),
     F.first("Year established").alias("Year established"),
     F.concat_ws(", ", F.collect_list("Properties")).alias("Properties"),
     F.concat_ws(", ", F.collect_list("Rooms")).alias("Rooms")  # Adjust based on your requirement
)
defaults = {
    "Headquarters": "Unknown Headquarters",
    "Locations": "Unknown Locations",
    "Year established": "Unknown Year",
    "Properties": "Unknown Properties",
    "Rooms": "unknown" 
df_first = df_first.fillna(defaults)
df_first.show()


+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+
|     Holding company|        Headquarters|           Locations|Year established|          Properties|               Rooms|
+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+
|               Accor|      Paris (France)|Worldwide, Worldw...|            1967|5,300[4], 5,300[4...|778,000[4], 778,0...|
|        Aman Resorts|   Baar, Switzerland|     20 countries[5]|            1988|               34[5]|          778,000[4]|
|           APA Group|       Tokyo (Japan)|  Japan, USA, Canada|            1988|355[6], 355[6], 3...|53,168[6], 53,168...|
|Archipelago Inter...|Jakarta, (Indonesia)|           Worldwide|            1997|                 150|           53,168[6]|
|Ascott Limited (The)|           Singapore|Worldwide (27 cou...|            1984|800[7], 800[7], 8...|135,000[7], 135,0...|
|       

24/09/26 17:18:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


4.1.4 cleaning information about references

In [46]:
from pyspark.sql.functions import  split,trim,regexp_replace
grouped_df = df_first.select(
         "Holding company",
    regexp_replace("Headquarters", r'\[\d+\]', "").alias("Headquarters"),
          regexp_replace("Locations", r'\[\d+\]', "").alias("Locations"),
           regexp_replace("Year established", r'\[\d+\]', "").alias("Year established"),
          regexp_replace("Properties", r'\[\d+\]', "").alias("Properties"),
          regexp_replace("Rooms", r'\[\d+\]', "").alias("Rooms")
       )
grouped_df.show()

24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+
|     Holding company|        Headquarters|           Locations|Year established|          Properties|               Rooms|
+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+
|               Accor|      Paris (France)|Worldwide, Worldw...|            1967|5,300, 5,300, 5,3...|778,000, 778,000,...|
|        Aman Resorts|   Baar, Switzerland|        20 countries|            1988|                  34|             778,000|
|           APA Group|       Tokyo (Japan)|  Japan, USA, Canada|            1988|       355, 355, 355|53,168, 53,168, 5...|
|Archipelago Inter...|Jakarta, (Indonesia)|           Worldwide|            1997|                 150|              53,168|
|Ascott Limited (The)|           Singapore|Worldwide (27 cou...|            1984|800, 800, 800, 80...|135,000, 135,000,...|
|       

4.1.5  create a new column associated "Headquarters country" 

In [47]:
from pyspark.sql.functions import  split,trim,regexp_replace
grouped_df = grouped_df.withColumn("City", split("Headquarters", r'\(')[0])
grouped_df = grouped_df.withColumn("Country", regexp_replace(split("Headquarters", r'\(')[1], r'\)', ''))
grouped_df.show()

+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+-----------------+--------------+
|     Holding company|        Headquarters|           Locations|Year established|          Properties|               Rooms|             City|       Country|
+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+-----------------+--------------+
|               Accor|      Paris (France)|Worldwide, Worldw...|            1967|5,300, 5,300, 5,3...|778,000, 778,000,...|           Paris |        France|
|        Aman Resorts|   Baar, Switzerland|        20 countries|            1988|                  34|             778,000|Baar, Switzerland|          NULL|
|           APA Group|       Tokyo (Japan)|  Japan, USA, Canada|            1988|       355, 355, 355|53,168, 53,168, 5...|           Tokyo |         Japan|
|Archipelago Inter...|Jakarta, (Indonesia)|           Worl

24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


4.2.1 Creating second Df

In [48]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, concat_ws, collect_set, regexp_replace, split, trim
spark = SparkSession.builder.appName("My App").getOrCreate()
df = spark.read.csv("/home/ai/Documents/Task_hotel/hotels.csv", header=True)
df2 = df.select("Holding company", "Category", "Managed brands", "Franchised brands")
df2.show()

24/09/26 17:18:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+---------------+--------+--------------------+--------------------+
|Holding company|Category|      Managed brands|   Franchised brands|
+---------------+--------+--------------------+--------------------+
|          Accor|  Luxury|             Raffles|Banyan Tree Hótel...|
|           NULL|    NULL|      Orient Express|                NULL|
|           NULL|    NULL|              Delano|                NULL|
|           NULL|    NULL|                 SO/|                NULL|
|           NULL|    NULL|      Sofitel Legend|                NULL|
|           NULL|    NULL|            Fairmont|                NULL|
|           NULL|    NULL|                 SLS|                NULL|
|           NULL|    NULL|             Sofitel|                NULL|
|           NULL|    NULL|The House of Orig...|                NULL|
|           NULL|    NULL|               Rixos|                NULL|
|           NULL|    NULL|         onefinestay|                NULL|
|           NULL| Premium|        

4.2.2 Cleaning DF

In [49]:
import pyspark.sql.functions as F
from pyspark.sql import Window
w_forward = Window.partitionBy().rowsBetween(Window.unboundedPreceding,Window.currentRow)
df2 = df2.select([F.last(df2[column], ignorenulls=True).over(w_forward).alias(column) for column in df2.columns])
df2.show()

+---------------+--------+--------------------+--------------------+
|Holding company|Category|      Managed brands|   Franchised brands|
+---------------+--------+--------------------+--------------------+
|          Accor|  Luxury|             Raffles|Banyan Tree Hótel...|
|          Accor|  Luxury|      Orient Express|Banyan Tree Hótel...|
|          Accor|  Luxury|              Delano|Banyan Tree Hótel...|
|          Accor|  Luxury|                 SO/|Banyan Tree Hótel...|
|          Accor|  Luxury|      Sofitel Legend|Banyan Tree Hótel...|
|          Accor|  Luxury|            Fairmont|Banyan Tree Hótel...|
|          Accor|  Luxury|                 SLS|Banyan Tree Hótel...|
|          Accor|  Luxury|             Sofitel|Banyan Tree Hótel...|
|          Accor|  Luxury|The House of Orig...|Banyan Tree Hótel...|
|          Accor|  Luxury|               Rixos|Banyan Tree Hótel...|
|          Accor|  Luxury|         onefinestay|Banyan Tree Hótel...|
|          Accor| Premium|        

24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:18:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


5.To specify the best type and nullable for each ncolumn

In [66]:
df2.printSchema()

root
 |-- Holding company: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Managed brands: string (nullable = true)
 |-- Franchised brands: string (nullable = true)



6.GroupBy func

6.1 Total number of rooms per "Headquarerd country"

In [36]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
grouped_df.show()
room_counts_by_country = grouped_df.groupBy("Country").agg({"Rooms": "sum"})
room_counts_by_country.show()

spark.stop()


24/09/26 17:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+-----------------+--------------+
|     Holding company|        Headquarters|           Locations|Year established|          Properties|               Rooms|             City|       Country|
+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+-----------------+--------------+
|               Accor|      Paris (France)|Worldwide, Worldw...|            1967|5,300, 5,300, 5,3...|778,000, 778,000,...|           Paris |        France|
|        Aman Resorts|   Baar, Switzerland|        20 countries|            1988|                  34|             778,000|Baar, Switzerland|          NULL|
|           APA Group|       Tokyo (Japan)|  Japan, USA, Canada|            1988|       355, 355, 355|53,168, 53,168, 5...|           Tokyo |         Japan|
|Archipelago Inter...|Jakarta, (Indonesia)|           Worl

24/09/26 17:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:16:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:16:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:16:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+----------+
|             Country|sum(Rooms)|
+--------------------+----------+
|              France|      NULL|
|               Japan|      NULL|
|           Indonesia|      NULL|
|            Pakistan|      NULL|
|             Bermuda|      NULL|
|                 USA|      NULL|
|               China|      NULL|
|            Thailand|      NULL|
|           Sri Lanka|      NULL|
|             Ireland|      NULL|
|      United Kingdom|      NULL|
|              Sweden|      NULL|
|               India|       5.0|
|              Canada|      NULL|
|                  UK|      NULL|
|United Arab Emirates|      NULL|
|             Germany|      NULL|
|  Dominican Republic|      NULL|
|               Korea|      NULL|
|               Spain|      NULL|
+--------------------+----------+
only showing top 20 rows



6.2 Average number of properties per "Headquarerd country"

In [53]:
avg_properties_by_country = grouped_df.groupBy("Country").agg({"Properties": "avg"})
avg_properties_by_country.show()

24/09/26 17:22:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:22:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:22:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:22:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+------------------+
|             Country|   avg(Properties)|
+--------------------+------------------+
|              France|              NULL|
|               Japan|              NULL|
|           Indonesia|             150.0|
|            Pakistan|               5.0|
|             Bermuda|              41.0|
|                 USA|52.666666666666664|
|               China|              NULL|
|            Thailand|              NULL|
|           Sri Lanka|              22.0|
|             Ireland|              NULL|
|      United Kingdom|              NULL|
|              Sweden|              39.0|
|               India|             106.0|
|              Canada|             126.0|
|                  UK|              NULL|
|United Arab Emirates|              24.0|
|             Germany|              NULL|
|  Dominican Republic|              10.0|
|               Korea|              NULL|
|               Spain|              NULL|
+--------------------+------------

6.3   Number of "Managed Brands" per "Holding Company" and "Category"

In [65]:
# df2.show()
managed_brands_per_holding_company_category = df2.groupBy("Holding company", "Category").agg({"Managed brands": "size"})
managed_brands_per_holding_company_category.show()

+--------------------+--------------+---------------------+
|     Holding company|      Category|count(Managed brands)|
+--------------------+--------------+---------------------+
|               Accor|        Luxury|                   11|
|               Accor|       Premium|                   13|
|               Accor|      Midscale|                    6|
|               Accor|       Economy|                    6|
|        Aman Resorts|        Luxury|                    1|
|           APA Group|      Midscale|                    3|
|Archipelago Inter...|      Midscale|                    1|
|Ascott Limited (The)|        Luxury|                    3|
|Ascott Limited (The)|       Premium|                    2|
|Ascott Limited (The)|       Upscale|                    3|
|Ascott Limited (The)|Upper-Midscale|                    2|
|Ascott Limited (The)|      Midscale|                    1|
|Ascott Limited (The)|Lower-Midscale|                    2|
|Ascott Limited (The)|       Economy|   

24/09/26 17:32:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:32:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:32:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/26 17:32:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
